<a href="https://colab.research.google.com/github/imran1386khan/Biobert-Training-Google-Colab/blob/master/GPT2_Train_MediAssist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pre-Requisite

In [4]:
import pandas as pd
import numpy as np
# !pip install tensorflow==2.0.0-alpha0

# !pip install tensorflow-gpu==1.14.0

%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow==2.0.0-alpha0
# !pip install tensorflow-gpu==1.14.0

     |████████████████████████████████| 79.9MB 47kB/s 
     |████████████████████████████████| 3.0MB 41.1MB/s 
     |████████████████████████████████| 419kB 39.4MB/s 


In [1]:
from tensorflow.python.client import device_lib
!cat /proc/meminfo | egrep "MemTotal*"

device_lib.list_local_devices()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

MemTotal:       13333556 kB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13182781225876456349, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12983343231269873471
 physical_device_desc: "device: XLA_CPU device"]

In [2]:
import tensorflow as tf
# print(tf.version)
!pip list | grep tensorflow

tensorflow               2.0.0a0             
tensorflow-addons        0.8.3               
tensorflow-datasets      2.1.0               
tensorflow-estimator     1.14.0              
tensorflow-gcs-config    2.1.8               
tensorflow-gpu           1.14.0              
tensorflow-hub           0.8.0               
tensorflow-metadata      0.22.0              
tensorflow-privacy       0.2.2               
tensorflow-probability   0.10.0              


In [17]:
#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

# !pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
# !pip install https://github.com/re-search/gpt2-estimator/archive/master.zip

!wget https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!wget https://github.com/re-search/gpt2-estimator/archive/master.zip

!unzip master.zip
!unzip v0.2.0_dev.zip

!mv DocProduct-0.2.0_dev/* /content/
!mv gpt2-estimator-master/* /content/
    
import os
import requests

--2020-05-29 10:40:32--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200529T104033Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJH%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQCFNXZzrJUR%2BQLyNKCEx1QhP5z6g24XSsbJWbtoBA0dCAIgHR9sLu9vYKDOFq4J%2BuCu4RSOIN8bssocALS%2FlIkswcYqvQMI6v%2F%2F%2F%2F%2F%2F%

In [0]:
!mkdir '/content/data'
!cp '/content/drive/My Drive/data/QandA_Dataset.csv' '/content/data/train_data.csv'

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
import os
if not os.path.exists('checkpoint'):
    os.mkdir('checkpoint')
    
if not os.path.exists('checkpoint/run1'):
    os.mkdir('checkpoint/run1')


import json
import requests
import urllib.request
import sys
import shutil
import re
from tqdm import tqdm, trange
import numpy as np
import tensorflow.compat.v1 as tf
import time
from datetime import datetime
import csv
import argparse

# if in Google Colaboratory
try:
    from google.colab import drive
except:
    pass

In [20]:
# if not os.path.exists('GPT2Folder'):
#     os.mkdir('GPT2Folder')

!cp -r '/content/drive/My Drive/GPT2Folder/' '/content/'
!cp -r '/content/GPT2Folder/GPT2Folder/BioBertFolder' '/content/'
!cp '/content/GPT2Folder/GPT2Folder/Float16EmbeddingsExpanded.pkl' '/content/'
  

# file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'

# urllib.request.urlretrieve( file_id, 'GPT2Folder/updatedGPTckpt.zip')

# import zipfile
# zip_ref = zipfile.ZipFile('GPT2Folder/updatedGPTckpt.zip', 'r')
# zip_ref.extractall('GPT2Folder')
# zip_ref.close()

cp: cannot stat '/content/GPT2Folder/GPT2Folder/BioBertFolder': No such file or directory
cp: cannot stat '/content/GPT2Folder/GPT2Folder/Float16EmbeddingsExpanded.pkl': No such file or directory


In [0]:
# !cp -r /content/GPT2Folder/* '/content/drive/My Drive/GPT2Folder/'
# !cp -r '/content/BioBertFolder' '/content/drive/My Drive/GPT2Folder/'

In [0]:
# urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

# if not os.path.exists('BioBertFolder'):
#     os.makedirs('BioBertFolder')
    
# import tarfile
# tar = tarfile.open("BioBert.tar.gz")
# tar.extractall(path='BioBertFolder/')
# tar.close()

# file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

# download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded.pkl')

In [21]:
def download_gpt2(model_name='117M'):
    """Downloads the GPT-2 model into the current directory
    from Google Cloud Storage.

    Adapted from https://github.com/openai/gpt-2/blob/master/download_model.py
    """

    subdir = os.path.join('models', model_name)
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    subdir = subdir.replace('\\', '/')  # needed for Windows

    for filename in ['checkpoint', 'encoder.json', 'hparams.json',
                     'model.ckpt.data-00000-of-00001', 'model.ckpt.index',
                     'model.ckpt.meta', 'vocab.bpe']:

        r = requests.get("https://storage.googleapis.com/gpt-2/" +
                         subdir + "/" + filename, stream=True)

        with open(os.path.join(subdir, filename), 'wb') as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(ncols=100, desc="Fetching " + filename,
                      total=file_size, unit_scale=True) as pbar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)

download_gpt2() # /content/models. ---> GPT2 (117M) model gets downloaded here


Fetching checkpoint: 1.00kit [00:00, 564kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 23.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 321kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 48.4Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.75Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 27.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 30.8Mit/s]                                                       


### Training GPT2 Model

In [0]:
d = pd.read_pickle('/content/GPT2Folder/Float16EmbeddingsExpanded.pkl')
d = d[:20000]
if not os.path.exists('/content/embed_data'):
    os.makedirs('/content/embed_data')
d.to_pickle('/content/embed_data/Float16EmbeddingsExpanded_short.pkl')
d = None

In [1]:
from docproduct.train_embedding_to_gpt2_data import train_embedding_to_gpt2_data

train_embedding_to_gpt2_data(
data_path='/content/embed_data/Float16EmbeddingsExpanded_short.pkl',
output_path='gpt2_train_data/bertffn_crossentropy_gpt2_train_data.zip',
number_samples=10,
batch_size=512,
search_by='answer')

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3

In [0]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [0]:
d = pd.read_parquet('/content/gpt2_train_data/bertffn_crossentropy_gpt2_train_data.zip')
d.head()

question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     answer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       question0                                                                                                                                                                                                                             

In [0]:
data = np.load('/content/gpt2_train_data/bertffn_crossentropy_gpt2_train_data_pretokenized.npy', allow_pickle=True)
print(data.shape)
print(data[0].shape)

FileNotFoundError: ignored

In [0]:
from gpt2_estimator.gpt2.src import model, sample, encoder
from gpt2_estimator.gpt2.src.load_dataset import load_dataset, Sampler #load_dataset or txt_load_dataset
from gpt2_estimator.gpt2.src.accumulate import AccumulatingOptimizer

from docproduct.train_gpt2 import train_gpt2

"""Function to train the GPT2 model

For each question, we use topk qa pairs that retreived by FAISS and the question
as features, and correct answer as target to train GPT2. 

Data: my eyes hurt, go see a doctor
Feature:
    question: aaa, answer: bbb, question: ccc, answer: ddd, question: my eyes hurt, answer:
Target: 
    go see a doctor


Keyword Arguments:
    model_dir {str} -- Path to save the GPT2 model (default: {'models/gpt2'})
    pretrained_path {str} -- Pretrained GPT2 model path,  (default: {'models/117M'})
    steps {int} -- Number of steps of training (default: {100000})
    batch_size {int} -- Batch size per GPU (default: {4})
    num_gpu {int} -- Number of GPU to use (default: {4})
    learning_rate {float} -- Learning rate (default: {0.0001})
"""
# Below function internally also invokes a function that creates features from data. 
# Default path is data_path='gpt2_train_data/bertffn_crossentropy_gpt2_train_data.zip'. --> This was created from train_embedding_to_gpt2_data file

train_gpt2(
        model_dir='models/gpt2',
        pretrained_path='models/117M',
        steps=100000,
        batch_size=1,
        max_seq_len=1024,
        num_gpu=2,
        learning_rate=0.0001)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_model_dir': 'models/gpt2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f90e4261b70>, '_device_fn': None, '_protocol': None, '_eval_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f90e4261b70>, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.serv

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/d

In [0]:
from gpt2_estimator.gpt2.src import model, sample, encoder
from gpt2_estimator.gpt2.src.load_dataset import load_dataset, Sampler #load_dataset or txt_load_dataset
from gpt2_estimator.gpt2.src.accumulate import AccumulatingOptimizer

from docproduct.train_gpt2 import train_gpt2

"""Function to train the GPT2 model

For each question, we use topk qa pairs that retreived by FAISS and the question
as features, and correct answer as target to train GPT2. 

Data: my eyes hurt, go see a doctor
Feature:
    question: aaa, answer: bbb, question: ccc, answer: ddd, question: my eyes hurt, answer:
Target: 
    go see a doctor


Keyword Arguments:
    model_dir {str} -- Path to save the GPT2 model (default: {'models/gpt2'})
    pretrained_path {str} -- Pretrained GPT2 model path,  (default: {'models/117M'})
    steps {int} -- Number of steps of training (default: {100000})
    batch_size {int} -- Batch size per GPU (default: {4})
    num_gpu {int} -- Number of GPU to use (default: {4})
    learning_rate {float} -- Learning rate (default: {0.0001})
"""
# Below function internally also invokes a function that does data massaging. 
# Default path is data_path='gpt2_train_data/bertffn_crossentropy_gpt2_train_data.zip'. --> This was created from train_embedding_to_gpt2_data file

train_gpt2(
        model_dir='models/gpt2',
        pretrained_path='models/117M',
        steps=100000,
        batch_size=1,
        max_seq_len=1024,
        num_gpu=1,
        learning_rate=0.0001)


INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_model_dir': 'models/gpt2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f2780f9f0f0>, '_device_fn': None, '_protocol': None, '_eval_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f2780f9f0f0>, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.serv

Pretokenization:   0%|          | 2/20000 [00:00<23:47, 14.01it/s]

`QUESTION: Please help me.[deleted] `ANSWER: Get checked for herpes yourself. Have you or your girlfriend ever had herpes on the lips (also known as cold sores)? The same virus can be transferred to the genitals (with fingers or directly), causing new symptoms there. `QUESTION: So my sister had this on her neck, we think it might be a bite or somethingage: 4approximate height: I think about 3 feetweight about 30gender Fmedications you take: none smoking status: Nope previous and current medical issues: none duration and location of complaint: neck, noticed todayhttp://imgur.com/VEACGHT `ANSWER: Could be a bite. It also looks a bit like a burn. If it's not bothering her, just leave it alone, maybe cover it with some vaseline or something. If it spreads or gets bigger or she develops more, it might be worth an appointment with her primary care physician.  `QUESTION: Do I have testicular cancer?!Hey, I am really scared that I have testicular cancer. I noticed a lump that just formed this 

Pretokenization:   0%|          | 9/20000 [00:00<18:21, 18.14it/s]

`QUESTION: I hit a electrified wire fence while dirtbiking..While riding a lil 50cc I rode into a electric wire fence, took one wire well across the chest just under my tits and now often whenever I bend over my left nostril (only left) drips super yellow snot, feels like a nosebleed but no blood. what did I do to myself? `ANSWER: When did this happen? `QUESTION: Hit my head, need some adviceI hit my head at a concert last night pretty hard, someone headbutted or elbowed me, i don't know.  I didn't black out or anything, no nausea.  I was drunk when it happened, and I still kept going after that.  (It was a metal concert so a lot of screaming, headbanging, and great things to do after you hit your head really hard.  I went home, went to bed, had a terrible dream,(unrelated), and woke up a couple times during the night feeling disoriented.  Woke up this morning, have a consistent headache and I guess dizziness and felt slow to respond to things, it's hard to describe.  It could just be 

...

Pretokenization: 100%|██████████| 20000/20000 [07:38<00:00, 43.62it/s]


Pretokenization successful!
Loading pretokenized data..
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Re

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1', '/replica:0/task:0/device:GPU:2').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1', '/replica:0/task:0/device:GPU:2').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1', '/replica:0/task:0/device:GPU:2').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1', '/replica:0/task:0/device:GPU:2').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU:0', '/replica:0/task:0/device:GPU:1', '/replica:0/task:0/device:GPU:2').
INFO:tensorflow:Reduce to /replica:0/task:0/device:GPU:0 then broadcast to ('/replica:0/task:0/device:GPU

TypeError: ignored

### Inference From GPT2 Model

In [0]:
from docproduct.predictor import GenerateQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
ffn_weight_file = None
bert_ffn_weight_file = 'GPT2Folder/models/bertffn_crossentropy/bertffn'
gpt2_weight_file = 'GPT2Folder/models/gpt2sync'
embedding_file = 'Float16EmbeddingsExpanded.pkl'

docGen = GenerateQADoc(pretrained_path=pretrained_path,
    ffn_weight_file=None,
    bert_ffn_weight_file=bert_ffn_weight_file,
    gpt2_weight_file = gpt2_weight_file,
    embedding_file=embedding_file)

def break_at_ignoring_whitespace(str, break_at):
    m = re.match(r"((\s*\w){%d})(.*)" % break_at, str, re.S)
    return (m.group(1), m.group(3)) if m else (str, '')

In [0]:
question_text = "My eyes get really dry when I sleep"  #@param {type:"string"}
search_similarity_by = 'answer'  #@param ['answer', "question"]
top_k=5 #@param {type:"number"}
number_of_answers_to_generate = 5 #@param {type:"number"}

# Break Sentence into 2 pieces of 1024 charaters and remaining 
inputText = break_at_ignoring_whitespace(question_text, 1024)

for i in range(number_of_answers_to_generate):
    advices = docGen.predict( inputText[0],
                  search_by=search_similarity_by, topk=top_k, answer_only=False)
    
    print("Answer # ", i+1)
    print(advices)
    print(' ')

In [0]:
question_text = "My eyes get really dry when I sleep"
def break_at_ignoring_whitespace(str, break_at):
    m = re.match(r"((\s*\w){%d})(.*)" % break_at, str, re.S)
    return (m.group(1), m.group(3)) if m else (str, '')

inputText = break_at_ignoring_whitespace(question_text, 3)

In [0]:
inputText

('My e', 'yes get really dry when I sleep')